# Natural Language to SQL with ChatGPT


## Motivation

The ability of LLMs to "understand" human language has made them widely accessible to the general public. Tools such as ChatGPT are becoming indispensable for many businesses. While these models excel at generating and refining text, they lack accuracy and rely on broad web data rather than specific business datasets. Custom GPTs can be trained on proprietary data, but this approach carries significant drawbacks, including privacy concerns and a tendency to hallucinate.

Accurate quantitative results can be retrieved directly from a database, but doing so requires knowledge of SQL and a deep understanding of the data model - skills that programmers typically have, but business users often lack. In fact, many software applications are developed solely to enable business users to query underlying databases.

A massive shortcut can be achieved by writing a program that uses OpenAI's API to convert a natural language inquiry into SQL and then executes the resulting query against the database. Since the results come from an actual database, they cannot be "hallucinated." There are still some concerns. The data model - though not the data - has to be shared. Furthermore, failures to generate accurate SQL can lead to frustration. Fine-tuning is crucial, but for now, we'll keep it simple.

## Prerequisites

- Access to OpenAI API 
- Basic Python Programming
- Basic SQL knowledge

Note that you need to sing up and obtain a token for the openai API. To use the API you must add some funds to the account. 
Running this tutorial will cost a few cents at most.

https://platform.openai.com/docs/overview

## Prepare to use Open AI

You must protect your API key by keeping it form appearing in plan view.
Create a .env file in the directory of this project and enter

```
OPENAI_API_KEY=<your key>
```

Use the following command to load the environment variables in the notebook.

In [24]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
from openai import OpenAI
openai_client = OpenAI()

response = openai_client.chat.completions.create(
        model ="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You answer every request with a Haiku."},
            {"role": "user", "content": "What time is it?" }
        ],
    )
response

ChatCompletion(id='chatcmpl-Aw9GlQHWSqhb2m75592NwEIbanjqX', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The clock ticks slowly  \nMoments slip through our fingers  \nEternal is now', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738422019, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=25, total_tokens=42, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [26]:
print(response.choices[0].message.content)

The clock ticks slowly  
Moments slip through our fingers  
Eternal is now


## Describe the database schema as a prompt to Chat GPT

I have provided a five table database that I adapted from this article: https://www.sqlitetutorial.net/sqlite-sample-database/

Here is function to convert the schema to a string:

In [27]:
from sqlalchemy import create_engine, MetaData, text

def build_schema(engine):
    """
    Read the schema from a database and convert it to a string
    :param engine: the sqlalchemy engine
    :return: the schema as a string
    """
    metadata = MetaData()
    metadata.reflect(bind=engine)
    tables = [f"{table.name}: {[col.name for col in table.columns]}" for table in metadata.tables.values()]
    return "\n".join(tables)

engine = create_engine("sqlite:///media_store.db")


In [28]:

# Here is the schema
schema = build_schema(engine)
schema.split("\n")

["album: ['album_id', 'title', 'artist']",
 "customer: ['customer_id', 'first_name', 'last_name', 'address', 'city', 'country', 'state', 'postal_code', 'email', 'phone']",
 "invoice: ['invoice_id', 'customer_id', 'billing_address', 'billing_city', 'billing_country', 'billing_postal_code', 'billing_state']",
 "invoice_item: ['invoice_line_id', 'invoice_id', 'track_id', 'quantity']",
 "track: ['track_id', 'name', 'album_id', 'composer', 'milliseconds', 'bytes', 'unit_price']"]

Let's define a helper method to execute a sql query and get the results as a Pandas data frame

In [29]:
import pandas as pd

def execute_sql_query(engine, query):
    """
    Execute a SQL query and return the results as a DataFrame
    :param engine: the sqlalchemy engine
    :param query: the SQL query
    :return: the results as a DataFrame
    """
    with engine.connect() as connection:
        results = connection.execute(text(query))
        return pd.DataFrame([row._mapping for row in results])


In [30]:
result = execute_sql_query(engine, """select i.billing_state as state, sum(t.unit_price*ii.quantity) as amount
from invoice_item ii 
inner join invoice i on i.invoice_id = ii.invoice_id 
inner join track t on t.track_id = ii.track_id 
where i.billing_country  = 'USA'
GROUP by state""")
result

,amount,state
0,37.62,AZ
1,115.86,CA
2,39.62,FL
3,43.62,IL
4,37.62,MA
5,37.62,NV
6,37.62,NY
7,47.62,TX
8,43.62,UT
9,39.62,WA


In [31]:
def build_system_prompt(schema_description):
    """
    Given a schema description, build the system prompt for the chatgpt API
    :param schema_description: the database schema as a string
    :return: the system prompt
    """
    instructions_prompt = """
    Given the following SQL tables, your job is to write SQLite queries to answer the user’s questions.
    Respond only with SQL and no additional content.
    """.strip()
    return f'{instructions_prompt}\n{schema_description}'

system_prompt = build_system_prompt(schema)
system_prompt

"Given the following SQL tables, your job is to write SQLite queries to answer the user’s questions.\n    Respond only with SQL and no additional content.\nalbum: ['album_id', 'title', 'artist']\ncustomer: ['customer_id', 'first_name', 'last_name', 'address', 'city', 'country', 'state', 'postal_code', 'email', 'phone']\ninvoice: ['invoice_id', 'customer_id', 'billing_address', 'billing_city', 'billing_country', 'billing_postal_code', 'billing_state']\ninvoice_item: ['invoice_line_id', 'invoice_id', 'track_id', 'quantity']\ntrack: ['track_id', 'name', 'album_id', 'composer', 'milliseconds', 'bytes', 'unit_price']"

In [32]:
def build_user_prompt(user_question):
    """
    Given a user question, build the user prompt for the chatgpt API
    :param user_question: the user question
    :return: the user prompt
    """
    return f"{user_question}\nSELECT"


In [33]:
def call_chatgpt(openai_client, system_prompt, user_prompt):
    """
    Call the chatgpt API to generate a SQL query
    :param openai_client: The openai clients
    :param system_prompt: The system prompt
    :param user_prompt: The user prompt
    :return: The response from the chatgpt API
    """
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
        max_tokens=156
    )
    return response

In [34]:
def extract_sql(response):
    """
    extract and format the SQL portion of the response
    :param response: the raw response from the chatgpt API
    :return: the SQL query
    """
    raw_query = response.choices[0].message.content.strip()
    if not raw_query.upper().startswith("SELECT"):
        raw_query = f'SELECT {raw_query}'
    return raw_query


In [35]:
def answer_user_query(engine, openai_client, user_query):
    """
    Answer a user query by generating SQL with chatgpt, executing the SQL,
    and returning the results as a DataFrame
    :param engine: the sqlalchemy engine
    :param openai_client: the openai client
    :param user_query: the user query
    :return: A pair containing the sql generated and the results as a DataFrame
    """
    schema = build_schema(engine)
    system_prompt = build_system_prompt(schema)
    user_prompt = build_user_prompt(user_query)
    chat_response = call_chatgpt(openai_client, system_prompt, user_prompt)
    print(chat_response)
    sql = extract_sql(chat_response)
    try :
        df = execute_sql_query(engine, sql)
    except Exception as e:
        print(f"Error executing SQL: {e}")
        return sql, "The assistant could not answer your question. Please try reformulating it."
    return sql, df


In [36]:
# Testing it out

openai_client = OpenAI()

user_question = "give me the total sales by territory"
sql, result = answer_user_query(engine, openai_client, user_question)


ChatCompletion(id='chatcmpl-Aw9HNSi26EFyjGvhPFwQ9UcctUbqq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='billing_country AS territory, SUM(unit_price * quantity) AS total_sales\nFROM invoice\nJOIN invoice_item ON invoice.invoice_id = invoice_item.invoice_id\nJOIN track ON invoice_item.track_id = track.track_id\nGROUP BY billing_country;', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738422057, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=49, prompt_tokens=175, total_tokens=224, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [37]:
result

,territory,total_sales
0,Argentina,37.62
1,Australia,37.62
2,Austria,42.62
3,Belgium,37.62
4,Brazil,190.10
5,Canada,303.96
6,Chile,46.62
7,Czech Republic,90.24
8,Denmark,37.62
9,Finland,41.62


In [38]:
sql.split('\n')

['SELECT billing_country AS territory, SUM(unit_price * quantity) AS total_sales',
 'FROM invoice',
 'JOIN invoice_item ON invoice.invoice_id = invoice_item.invoice_id',
 'JOIN track ON invoice_item.track_id = track.track_id',
 'GROUP BY billing_country;']

In [39]:
# A more complex query

user_query = "Find the top ten album artists that made the most sales on tracks to customers in CA"
sql, df = answer_user_query(engine, openai_client, user_query)


ChatCompletion(id='chatcmpl-Aw9HgH2nhfNcmXWF8ysPVAKAPPr6d', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="artist, SUM(unit_price * quantity) AS total_sales\nFROM album\nJOIN track ON album.album_id = track.album_id\nJOIN invoice_item ON track.track_id = invoice_item.track_id\nJOIN invoice ON invoice_item.invoice_id = invoice.invoice_id\nJOIN customer ON invoice.customer_id = customer.customer_id\nWHERE customer.state = 'CA'\nGROUP BY artist\nORDER BY total_sales DESC\nLIMIT 10;", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1738422076, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=84, prompt_tokens=185, total_tokens=269, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=Pro

In [40]:
sql.split('\n')

['SELECT artist, SUM(unit_price * quantity) AS total_sales',
 'FROM album',
 'JOIN track ON album.album_id = track.album_id',
 'JOIN invoice_item ON track.track_id = invoice_item.track_id',
 'JOIN invoice ON invoice_item.invoice_id = invoice.invoice_id',
 'JOIN customer ON invoice.customer_id = customer.customer_id',
 "WHERE customer.state = 'CA'",
 'GROUP BY artist',
 'ORDER BY total_sales DESC',
 'LIMIT 10;']

In [41]:
df

,artist,total_sales
0,Iron Maiden,19.80
1,Eric Clapton,8.91
2,The Office,5.97
3,Van Halen,5.94
4,Metallica,5.94
5,Deep Purple,5.94
6,Antônio Carlos Jobim,5.94
7,Toquinho & Vinícius,4.95
8,The Cult,4.95
9,Red Hot Chili Peppers,4.95


## The Future

The paradigm for building data-driven software applications is shifting dramatically. Instead of training users to obtain information by using buttons, dropdowns, sliders, links, and all that jazz, they can instead simply state what they want. Whereas in this particular example, ChatGPT just generates SQL, it can write code in any language, perform calculations, draw diagrams, summarize data, and perform countless other operations.